In [8]:
import os
import glob
import pandas as pd

In [9]:
path = 'output*.txt'
files = glob.glob(path)
files

['outputMongoDB_workloada.txt',
 'outputMongoDB_workloadb.txt',
 'outputMongoDB_workloadd.txt',
 'outputMongoDB_workloadf.txt',
 'outputMYSQL_workloada.txt',
 'outputMYSQL_workloadb.txt',
 'outputMYSQL_workloadd.txt',
 'outputMYSQL_workloadf.txt']

In [37]:
# following code taken from Mahender who shared it in the whatsapp group

def read_ycsb(in_file, out_name):
    out_file = out_name+'.csv'
    with open(in_file) as f:
        lines = f.readlines()
    
    data = {
        #'operation': [],
        'index': out_name,
        'insert average_latency': [],
        'insert 95_latency': [],
        'insert 99_latency': [],
        'cleanup average_latency': [],
        'cleanup 95_latency': [],
        'cleanup 99_latency': [],
        'overall runtime(ms)': [],
        'overall throughput(ops/sec)':[]
    }
    
    for line in lines:
        if line.startswith('[INSERT]'):
            parts = line.strip().split()
            if parts[1] == 'AverageLatency(us),':
                data['insert average_latency'].append(float(parts[2]))
            if parts[1] == '95thPercentileLatency(us),':
                data['insert 95_latency'].append(float(parts[2]))
            if parts[1] == '99thPercentileLatency(us),':
                data['insert 99_latency'].append(float(parts[2]))
        if line.startswith('[CLEANUP]'):
            parts = line.strip().split()
            if parts[1] == 'AverageLatency(us),':
                data['cleanup average_latency'].append(float(parts[2]))
            if parts[1] == '95thPercentileLatency(us),':
                data['cleanup 95_latency'].append(float(parts[2]))
            if parts[1] == '99thPercentileLatency(us),':
                data['cleanup 99_latency'].append(float(parts[2]))
        if line.startswith('[OVERALL]'):
            parts = line.strip().split()
            if parts[1] == 'RunTime(ms),':
                data['overall runtime(ms)'].append(float(parts[2]))
            if parts[1] == 'Throughput(ops/sec),':
                data['overall throughput(ops/sec)'].append(float(parts[2]))
            
    dfR = pd.DataFrame.from_dict(data)
    dfR.to_csv(out_file)
    return dfR                     

In [42]:
df_list = []

for file in files:
    out_name = file[:-4]
    df_list.append(read_ycsb(file, out_name))

In [43]:
df = pd.concat(df_list)

In [46]:
df.set_index('index', inplace=True)

In [47]:
df

,insert average_latency,insert 95_latency,insert 99_latency,cleanup average_latency,cleanup 95_latency,cleanup 99_latency,overall runtime(ms),overall throughput(ops/sec)
index,,,,,,,,
outputMongoDB_workloada,877.630,1289.0,4021.0,3119.0,3119.0,3119.0,2104.0,475.285171
outputMongoDB_workloadb,964.210,1084.0,4191.0,2929.0,2929.0,2929.0,2102.0,475.737393
outputMongoDB_workloadd,862.660,1145.0,4543.0,2981.0,2981.0,2981.0,2080.0,480.769231
outputMongoDB_workloadf,812.369,1185.0,1977.0,3239.0,3239.0,3239.0,1890.0,529.100529
outputMYSQL_workloada,6285.116,10343.0,14735.0,2395.0,2395.0,2395.0,7794.0,128.303823
outputMYSQL_workloadb,6894.304,9863.0,14927.0,2161.0,2161.0,2161.0,8304.0,120.423892
outputMYSQL_workloadd,7345.666,10471.0,16431.0,2199.0,2199.0,2199.0,8684.0,115.154307
outputMYSQL_workloadf,6289.398,10439.0,15071.0,2347.0,2347.0,2347.0,7680.0,130.208333


Workload A: Update heavy workload
Application example: Session store recording recent actions
Read/update ratio: 50/50

Workload B: Read mostly workload
Application example: photo tagging; add a tag is an update, but most operations are to read tags
Read/update ratio: 95/5

Workload D: Read latest workload
Application example: user status updates; people want to read the latest
Read/update/insert ratio: 95/0/5

Workload F: Read-modify-write workload
Application example: user database, where user records are read and modified by the user or to record user activity             
Read/read-modify-write ratio: 50/50

Conclusion: MongoDB is far outperforming MySQL with regards to inserts. It's similar for cleanup. It has a much faster runtime and can perform approximately four times as many operations per second. MongoDB wins.